### [KOR]
- <span style = 'font-size:1.2em;line-height:1.5em'><b>3. </b>아래 데이터는 konlpy에서 제공하는 naver영화 평점 데이터이다. 아래 데이터를 보고, document column에 있는 text들을 전처리해보세요.</span>

    - <span style = 'font-size:1.1em;line-height:1.5em'> 띄어쓰기, 오타 교정, 형태소 분석기를 활용한 tokenizing, normalizing, word_dictionary 생성(빈도수가 상위 500개), Integer encoding, DTM 생성, TF-IDF생성</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'> DTM, TF-IDF의 각 row vector는 500차원의 벡터이다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'> 만약 Memory Error가 발생한다면 차원수를 더 줄여도 된다.</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'> DTM이나 TF-IDF를 생성할 때, OOV문제는 무시하여 벡터에 반영하지 않아도 된다.</span>

- 오타 교정 불가 (hanspell 이용 불가, selenium 시간 이슈로 불가)
- 띄어쓰기 불가 (Kospacing 속도 이슈로 불가)

In [1]:
from selenium import webdriver
from pykospacing import Spacing
import pickle
import pandas as pd
import numpy as np
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from konlpy.tag import Okt
import requests
import json
from hanspell import spell_checker
import time
from selenium.webdriver.common.by import By
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x2725a876910>)

In [3]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [4]:
train_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_data['document'] = train_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['document'].replace('', np.nan, inplace=True) # document column이 비어있다면(공백), Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거
print('전처리 후 학습용 샘플의 개수 :',len(train_data))

C:\Users\user\AppData\Local\Temp\ipykernel_25552\1209056594.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
C:\Users\user\AppData\Local\Temp\ipykernel_25552\1209056594.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경


전처리 후 학습용 샘플의 개수 : 145393


In [5]:
documents = train_data['document'].tolist()

In [6]:
okt = Okt()

In [7]:
lemmantized_list =[]
for i in range(len(documents)):
    lemmantized_list.append(okt.morphs(documents[i], stem=True)) # lemmantization

In [8]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords=[x.replace('\n','') for x in stopwords]

In [9]:
removal_stopwords = []
for i in range(len(lemmantized_list)):
    for j in lemmantized_list[i]:
        if j not in stopwords:
            removal_stopwords.append(j)

In [10]:
cnt = Counter(removal_stopwords)

In [11]:
vocab = cnt.most_common(500)

In [12]:
word_dict = dict()
for i, (word, freq) in enumerate(vocab):
    word_dict[word] = i

In [13]:
vector = CountVectorizer(vocabulary=word_dict)
results = vector.fit_transform(documents).toarray()
results

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
vector = TfidfVectorizer(vocabulary=word_dict)
tfidf = vector.fit_transform(documents).toarray()
tfidf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.68125303, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])